In [22]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from functools import reduce
import re
import matplotlib.pyplot as plt
import datetime

In [43]:
df_main=pd.read_csv('data.csv')
#df_main=df_main.drop['Unamed']
df_main

,Unnamed: 0,Date,ST,CB,PB,GO,CA
0,0,2020-01-01,194.59,79.43,259.95,161.41,96.39
1,1,2020-01-02,194.59,77.58,253.28,143.95,96.85
2,2,2020-01-03,194.12,78.03,255.12,145.86,96.84
3,3,2020-01-06,193.81,77.50,254.16,147.39,96.67
4,4,2020-01-07,194.30,77.73,254.36,147.97,97.00
...,...,...,...,...,...,...,...
279,279,2020-12-25,194.59,79.43,259.95,161.41,90.32
280,280,2020-12-28,204.02,80.86,253.55,175.71,90.34
281,281,2020-12-29,204.00,80.86,253.82,176.35,89.99
282,282,2020-12-30,203.32,80.42,253.84,177.70,89.68


In [49]:
df_main=df_main.drop('Unnamed: 0', axis=1)

In [50]:
df_main

,Date,ST,CB,PB,GO,CA
0,2020-01-01,194.59,79.43,259.95,161.41,96.39
1,2020-01-02,194.59,77.58,253.28,143.95,96.85
2,2020-01-03,194.12,78.03,255.12,145.86,96.84
3,2020-01-06,193.81,77.50,254.16,147.39,96.67
4,2020-01-07,194.30,77.73,254.36,147.97,97.00
...,...,...,...,...,...,...
279,2020-12-25,194.59,79.43,259.95,161.41,90.32
280,2020-12-28,204.02,80.86,253.55,175.71,90.34
281,2020-12-29,204.00,80.86,253.82,176.35,89.99
282,2020-12-30,203.32,80.42,253.84,177.70,89.68


In [112]:
def month_price(n_months,date_in):
    '''
    #set date
    date_in = (date_in.replace(day=day_number) + datetime.timedelta(days=32)).replace(day=day_number)
    print(date_in)
    #cycle and store the price of the different periods in a matrix
    '''
    delta = datetime.timedelta(days=1)
    frames=[]
    for i in range(n_months):
        filtered_df = df_main.loc[df_main['Date'] == str(date_in)]
        while (filtered_df.index.size < 1):
            filtered_df = df_main.loc[df_main['Date'] == str(date_in)]
            date_in+=delta

        frames.append(filtered_df)
        #date_in = (date_in.replace(day=day_number) + datetime.timedelta(days=32)).replace(day=day_number)
    return frames

## BUY

In [64]:
money_in=100
allocation=np.array([.5, .2, .2, .1, 0])
date_in="2020-01-01"

value_t0=money_in*allocation


filtered_df = df_main.loc[df_main['Date'] == date_in]
filtered_df=filtered_df.to_numpy()

price=np.array([filtered_df[0,2],filtered_df[0,3],filtered_df[0,4],filtered_df[0,5],1])

shares=value_t0/price




data={'Asset':['ST', 'CB', 'PB', 'GO', 'CA'],
     'Amount':value_t0,
     'Asset price':price,
     '#shares':shares}



one_off_investment = pd.DataFrame(data,columns=['Asset',  'Amount',  'Asset price', '#shares'])

print(one_off_investment)
print(date_in)

  Asset  Amount  Asset price   #shares
0    ST    50.0        79.43  0.629485
1    CB    20.0       259.95  0.076938
2    PB    20.0       161.41  0.123908
3    GO    10.0        96.39  0.103745
4    CA     0.0         1.00  0.000000
2020-01-01


## REBALANCING

In [119]:
# periods in months 1.3.6.9.12

date_in="2020-01-01"

date_in=datetime.date(2020,1,1)#example

'''
price=month_price(1,date_in,15)
price=price[0]
print(price)
price=price.iloc[:,1:] #solo perchè sto lavorando con data che ho esportato precedentemente
price_out=price
'''


if date_in.day>=15:
    date_in = (date_in.replace(day=15) + datetime.timedelta(days=32)).replace(day=15)
else:
    date_in = (date_in.replace(day=15))
    
price=month_price(1,date_in)  
price=price[0]
price_out=price.iloc[:,1:]
#print(price_out)

percentage_t0=allocation#np.array([50, 20, 0, 30, 0])#  

value_t1=np.multiply(shares, price_out)


sum_value_t1=float(value_t1.sum(axis=1))#103,1

percentage_t1=((value_t1*100)/sum_value_t1)/100
#print("percentage_t0: ",percentage_t0)
#print("percentage_t1: ", percentage_t1)

value_tr2=value_t1*percentage_t0/percentage_t1

#print("value tr2: ",value_tr2)
sum_value_tr2=float(value_tr2.sum(axis=1))#103,1

percentage_tr2=((value_tr2*100)/sum_value_tr2)/100
#print("percentage tr2: ",percentage_tr2)

data={'Asset':['ST', 'CB', 'PB', 'GO', 'CA'],
     '$_t0':value_t0,
     '%_t0':percentage_t0,
     '$_t1':value_t1.iloc[0],
     '%_t1':percentage_t1.iloc[0],
     '$_tr2':value_tr2.iloc[0],
     '%_tr2':percentage_tr2.iloc[0],
      }

portfolio_rebalancing=pd.DataFrame(data,columns=['$_t0','%_t0','$_t1','%_t1','$_tr2','%_tr2'])
print( portfolio_rebalancing)
#print(date_in)
#it works actually but we got some problem with cash 


    $_t0  %_t0        $_t1      %_t1      $_tr2  %_tr2
ST  50.0   0.5  123.863779  0.701610  88.271084    0.5
CB  20.0   0.2    6.038854  0.034206  35.308434    0.2
PB  20.0   0.2   31.436714  0.178069  35.308434    0.2
GO  10.0   0.1   15.202822  0.086114  17.654217    0.1
CA   0.0   0.0    0.000000  0.000000        NaN    NaN
